# <b style="color: white; background-color: #00bbff; padding: 5px 10px; border-radius: 5px;">INSTALL LIBRARY</b>

In [ ]:
!pip install faker

# <b style="color: white; background-color: #00bbff; padding: 5px 10px; border-radius: 5px;">INSERT DATA INTO CUSTOMER TABLE</b>

In [ ]:
from faker import Faker
import psycopg2
from tqdm import tqdm
import random

# Inicializa o Faker
fake = Faker('pt_BR')  # Locale brasileiro

# Conexão com o banco
conn = psycopg2.connect(
    dbname="postgres",
    user="postgres",
    password="admin",
    host="192.168.0.202",
    port="5432"
)
cursor = conn.cursor()

# Configurações
#total_registros = 100_000
#batch_size = 10_000  # Inserções em lote para performance
total_registros = 1
batch_size = 1  # Inserções em lote para performance
status_opcoes = ['ativo', 'inativo', 'pendente']

def gerar_dados_fake(inicio, qtd):
    dados = []
    for i in range(inicio, inicio + qtd):
        nome = fake.name()
        email = f"{nome.lower().replace(' ', '.')}.{i}@exemplo.com"
        data_cadastro = fake.date_between(start_date='-2y', end_date='today')
        status = random.choice(status_opcoes)
        dados.append((nome, email, data_cadastro, status))
    return dados

# Inserção em lotes
for inicio in tqdm(range(0, total_registros, batch_size)):
    dados = gerar_dados_fake(inicio, batch_size)
    args_str = ','.join(cursor.mogrify("(%s,%s,%s,%s)", x).decode("utf-8") for x in dados)
    cursor.execute(f"INSERT INTO clientes (nome, email, data_cadastro, status) VALUES {args_str}")
    conn.commit()

cursor.close()
conn.close()


# <b style="color: white; background-color: #00bbff; padding: 5px 10px; border-radius: 5px;">INSERT DATA INTO SALES TABLE/b>

In [ ]:
import psycopg2
from faker import Faker
import random
from tqdm import tqdm
from datetime import datetime, timedelta

# Configurações de conexão (substitua com suas credenciais)
# Conexão com o banco
conn = psycopg2.connect(
    dbname="postgres",
    user="postgres",
    password="admin",
    host="192.168.0.202",
    port="5432"
)
cursor = conn.cursor()

fake = Faker('pt_BR')

# Configurações
#total_vendas = 550_000  # 550 mil registros
#batch_size = 10_000     # Tamanho do lote para inserção
total_vendas = 1  # 550 mil registros
batch_size = 1     # Tamanho do lote para inserção
produtos = [
    {'nome': 'Notebook', 'preco': 3500.00, 'categoria': 'eletronicos'},
    {'nome': 'Smartphone', 'preco': 2200.00, 'categoria': 'eletronicos'},
    {'nome': 'Tablet', 'preco': 1500.00, 'categoria': 'eletronicos'},
    {'nome': 'Fone Bluetooth', 'preco': 250.00, 'categoria': 'acessorios'},
    {'nome': 'Smartwatch', 'preco': 800.00, 'categoria': 'eletronicos'},
    {'nome': 'Mouse Gamer', 'preco': 180.00, 'categoria': 'perifericos'},
    {'nome': 'Teclado Mecânico', 'preco': 300.00, 'categoria': 'perifericos'},
    {'nome': 'Monitor 24"', 'preco': 1200.00, 'categoria': 'eletronicos'},
    {'nome': 'Cadeira Gamer', 'preco': 1500.00, 'categoria': 'mobilia'},
    {'nome': 'Mesa Escritório', 'preco': 800.00, 'categoria': 'mobilia'}
]
status_venda = ['concluida', 'cancelada', 'devolvida', 'pendente']

def gerar_vendas_fake(qtd_vendas):
    # Primeiro obtemos os IDs de clientes existentes
    cursor.execute("SELECT id FROM clientes")
    ids_clientes = [row[0] for row in cursor.fetchall()]
    
    if not ids_clientes:
        raise Exception("Nenhum cliente encontrado na tabela clientes. Execute primeiro o script de geração de clientes.")
    
    vendas = []
    for _ in range(qtd_vendas):
        cliente_id = random.choice(ids_clientes)
        produto = random.choice(produtos)
        quantidade = random.randint(1, 3)
        preco_unitario = produto['preco'] * (1 - random.uniform(0, 0.2))  # Aplica desconto de até 20%
        total = preco_unitario * quantidade
        data_venda = fake.date_time_between(start_date='-2y', end_date='now')
        status = random.choice(status_venda)
        
        vendas.append((
            cliente_id,
            produto['nome'],
            produto['categoria'],
            quantidade,
            preco_unitario,
            total,
            data_venda,
            status
        ))
    return vendas

# Criação da tabela de vendas (se não existir)
cursor.execute("""
CREATE TABLE IF NOT EXISTS vendas (
    id SERIAL PRIMARY KEY,
    cliente_id INTEGER REFERENCES clientes(id),
    produto VARCHAR(100),
    categoria VARCHAR(50),
    quantidade INTEGER,
    preco_unitario DECIMAL(10,2),
    total DECIMAL(10,2),
    data_venda TIMESTAMP,
    status VARCHAR(20),
    data_processamento TIMESTAMP DEFAULT CURRENT_TIMESTAMP
)
""")
conn.commit()

# Inserção em lotes
print(f"Gerando {total_vendas} registros de vendas...")
for inicio in tqdm(range(0, total_vendas, batch_size)):
    qtd = min(batch_size, total_vendas - inicio)
    vendas = gerar_vendas_fake(qtd)
    
    args_str = ','.join(cursor.mogrify("(%s,%s,%s,%s,%s,%s,%s,%s)", x).decode("utf-8") for x in vendas)
    cursor.execute(f"""
        INSERT INTO vendas 
        (cliente_id, produto, categoria, quantidade, preco_unitario, total, data_venda, status) 
        VALUES {args_str}
    """)
    conn.commit()

cursor.close()
conn.close()

print("Dados de vendas gerados com sucesso!")